# Using 16O/18O exchange for enhanced LC-MS identification


Import libraries. For correct work we need rdkit v. 2020.09.1, PubChemPy v.1.0.4. 
File O18.py includes functions to calculate max number of exchanges
File Isomers.py includes functions to retrieve and clean isomers list
File Fragmentation trees includes functions to calculate a tree

In [1]:
from O18 import *
from isomers import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from rdkit.Chem import Draw
from tqdm import tqdm
tqdm.pandas()


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Insert formula here

In [2]:
#formula = 'C16H14O3' #ketoprofen
#formula = 'C19H28O2' #testosterone
#formula = 'C20H21O4Cl' #Fenofibrate
#formula = "C16H13N3O3" #mebendazole
#formula = "C25H34O6" #budesonide
formula = 'C19H16O4' #warfarin

In [3]:
isomers = get_isomers_for_formula(formula)

Insert experimentally observed number of exchanges here

In [4]:
N = 1

In [5]:
filtered  = clean_O18_MS1(isomers, N, formula)

In [6]:
filtered

,InChI,N
0,InChI=1S/C19H16O4/c1-12(20)11-15(13-7-3-2-4-8-...,1
1,InChI=1S/C19H16O4/c20-16-7-1-14(2-8-16)5-11-18...,2
3,InChI=1S/C19H16O4/c20-16-7-1-14(2-8-16)5-11-18...,1
6,InChI=1S/C19H16O4/c1-3-23-19(21)17-16(12-7-5-4...,1
7,InChI=1S/C19H16O4/c1-12-8-16(22-11-13(2)20)19-...,1
...,...,...
1605,InChI=1S/C19H16O4/c20-18(21)16-9-5-4-8-15(16)1...,4
1606,InChI=1S/C19H16O4/c1-11(19(21)22)17-13-7-3-5-9...,2
1607,InChI=1S/C19H16O4/c1-2-22-17-12-11-14(13-7-3-4...,2
1608,InChI=1S/C19H16O4/c1-2-7-23-19(22)17-11-15-9-1...,2


Here we can draw grid of all the matched candidates

In [7]:
filtered['mol'] = filtered.InChI.apply(MolFromInchi) 
filtered['N_str'] = filtered.N.apply(str) 
#img=Draw.MolsToGridImage(list(filtered.mol)[:50],molsPerRow=2,subImgSize=(600,600), legends=list(filtered.N_str)[:50]) 
#img #uncomment to get the image

/home/pc-1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/pc-1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Here we will try to map fragmentation with the remaining candidates

In [8]:
from rdkit import Chem
from rdkit.Chem.inchi import *
from pyparsing import * #required for parse_formula
from rdkit.Chem import rdmolops

In [9]:
class fragments():
    def __init__(self, formula):
        self.formula = formula  
        self.parsed_formula=self.parse_formula()
       
       
    def parse_formula(self):
        elements = [(1, "H"),(2, "He"),(3, "Li"),(4, "Be"),(5, "B"),(6, "C"),(7, "N"),(8, "O"),(9, "F"),(10, "Ne"),(11, "Na"),(12, "Mg"),(13, "Al"),(14, "Si"),(15, "P"),(16, "S"),(17, "Cl"),(18, "Ar"),(19, "K"),(20, "Ca"),(21, "Sc"),(22, "Ti"),(23, "V"),(24, "Cr"),(25, "Mn"),(26, "Fe"),(27, "Co"),(28, "Ni"),(29, "Cu"),(30, "Zn"),(31, "Ga"),(32, "Ge"),(33, "As"),(34, "Se"),(35, "Br"),(36, "Kr"),(37, "Rb"),(38, "Sr"),(39, "Y"),(40, "Zr"),(41, "Nb"),(42, "Mo"),(43, "Tc"),(44, "Ru"),(45, "Rh"),(46, "Pd"),(47, "Ag"),(48, "Cd"),(49, "In"),(50, "Sn"),(51, "Sb"),(52, "Te"),(53, "I"),(54, "Xe"),(55, "Cs"),(56, "Ba"),(57, "La"),(58, "Ce"),(59, "Pr"),(60, "Nd"),(61, "Pm"),(62, "Sm"),(63, "Eu"),(64, "Gd"),(65, "Tb"),(66, "Dy"),(67, "Ho"),(68, "Er"),(69, "Tm"),(70, "Yb"),(71, "Lu"),(72, "Hf"),(73, "Ta"),(74, "W"),(75, "Re"),(76, "Os"),(77, "Ir"),(78, "Pt"),(79, "Au"),(80, "Hg"),(81, "Tl"),(82, "Pb"),(83, "Bi"),(84, "Po"),(85, "At"),(86, "Rn"),(87, "Fr"),(88, "Ra"),(89, "Ac"),(90, "Th"),(91, "Pa"),(92, "U"),(93, "Np"),(94, "Pu"),(95, "Am"),(96, "Cm"),(97, "Bk"),(98, "Cf"),(99, "Es"),(100, "Fm"),(101, "Md"),(102, "No"),(103, "Lr"),(104, "Rf"),(105, "Db"),(106, "Sg"),(107, "Bh"),(108, "Hs"),(109, "Mt"),(110, "Ds"),(111, "Rg"),(112, "Cn"),(113, "Uut"),(114, "Uuq"),(115, "Uup"),(116, "Uuh"),(118, "Uuo")]
        el_list = sorted(list(zip(*elements))[1],key=lambda x:len(x),reverse=True)
        el = Or([Word(x) for x in el_list])
        isotop = el ^ Group(Suppress("[") + OneOrMore(Word(nums)) + el + Suppress("]")).setParseAction(lambda str, location, tokens: "".join(tokens[0]))
        parser = OneOrMore(Group(isotop + ZeroOrMore(Word(nums))))
        parsed = parser.parseString(self.formula)
        parsed_formula={}
        for p in parsed:
            if p[0] =='H': continue
            if len(p) == 2:
                parsed_formula[p[0]] = int(p[1])
            else:
                parsed_formula[p[0]]=1
        return parsed_formula

In [44]:
class mapped_fragments():
    def __init__(self, inchi, fragment_formulas, labeled, N_labels):
        if labeled ==False:
            self.mol_list = [MolFromInchi(inchi)]
        else:
            self.mol_list =run_16O18O_exchange (inchi, N_labels)
    
        self.fragments = []
        for formula in fragment_formulas:
            frag = fragments(formula)
            self.fragments.append(frag.parsed_formula)
        self.number_of_fragments = len(fragment_formulas)
        self.number_of_mapped_fragments = self.map_fragments()
        
        
    @staticmethod        
    def number_atoms(parsed_formula):
        len=0
        for n in parsed_formula.values():
            len+=n
        return len  
    
    def transform_bond_to_atom_subgraph(self,mol, bond_subgraph):
        atom_subgraph=set()
        bond_subgraph = list(bond_subgraph)
        for bond in bond_subgraph:
            atom_subgraph.add(mol.GetBondWithIdx(bond).GetBeginAtomIdx())
            atom_subgraph.add(mol.GetBondWithIdx(bond).GetEndAtomIdx())
        return atom_subgraph
    
    
    def get_formula_dict_for_subgraph(self, mol, atom_subgraph):
        formula_dict={}
        for atom_number in atom_subgraph:
            atom = mol.GetAtomWithIdx(atom_number)

            if atom.GetIsotope()!=0:
                atom_symb =str(atom.GetIsotope())+atom.GetSymbol()
            else:
                atom_symb = atom.GetSymbol()
            if atom_symb not in formula_dict.keys():
                formula_dict[atom_symb] = 1
            else:
                formula_dict[atom_symb] += 1
        return formula_dict
    def get_all_subgraphs_of_lengthN(self, mol, N):
        all_subgraphs_of_lengthN = set()
        all_formula_dict = []
        all_bond_subgraphs = rdmolops.FindAllSubgraphsOfLengthMToN(mol, 1, N)
        for bond_subgraphs in all_bond_subgraphs:
            for bond_subgraph in bond_subgraphs:
                atom_subgraph = self.transform_bond_to_atom_subgraph(mol, bond_subgraph)
                if len(atom_subgraph)==N:
                    all_subgraphs_of_lengthN.add(tuple(atom_subgraph))

        for atom_subgraph in all_subgraphs_of_lengthN:
            all_formula_dict.append(self.get_formula_dict_for_subgraph(mol, atom_subgraph))

        return all_formula_dict
    
    
    
    def check_formula(self,mol, parsed_formula, N):
        all_formula_dict = self.get_all_subgraphs_of_lengthN(mol, N)
        for formula in all_formula_dict:
            if parsed_formula==formula:
                return 1
        return 0
    def map_fragments(self):
        checked_all_mols=[]
        for mol in self.mol_list:
            checked = []
            for formula in self.fragments:
                checked.append(self.check_formula(mol, formula, self.number_atoms(formula)))
            checked_all_mols.append(sum(checked))
            try:
                return max(checked_all_mols)
            except:
                return 0

In [11]:
inchi ='InChI=1S/C16H13N3O3/c1-22-16(21)19-15-17-12-8-7-11(9-13(12)18-15)14(20)10-5-3-2-4-6-10/h2-9H,1H3,(H2,17,18,19,21)'

In [12]:
def calculate_mapped_frags(inchi, frags, labeled, N_labels):
    return mapped_fragments(inchi, frags, labeled, N_labels).map_fragments()

In [32]:
'''
#ketoprofen
fragsO18=["C15H13[18O]", "C14H10[18O]","C10H9[18O]3", "C9H7[18O]", "C7H5[18O]"]
frags=["C15H13O", "C14H10O","C10H9O3", "C9H7O", "C7H5O"]

#testosteroe
fragsO18=["C19H27[18O]", "C19H25", "C8H11[18O]","C7H9[18O]", "C6H9[18O]"]
frags=["C19H27O", "C19H25", "C8H11O","C7H9O", "C6H9O"]



#fenobirate
fragsO18=["C13H10[18O]Cl", "C7H4[18O]Cl", "C7H5[18O]O"]
frags=["C13H10OCl", "C7H4OCl", "C7H5O2"]


#budesonide
fragsO18=["C25H33[18O]2O3", "C25H31[18O]2O2", "C21H25[18O]2O2", "C21H23[18O]O2", "C12H13[18O]", "C10H11[18O]"]
frags=["C25H33O5", "C25H31O4", "C21H25O4", "C21H23O3", "C12H13O", "C10H11O"]



#mebendazole
fragsO18=["C15H10N3[18O]O", "C9H4N3[18O]O", "C7H5[18O]"]
frags=["C15H10N3O2", "C9H4N3O2", "C7H5O"]
'''
#warfarin
fragsO18=["C19H15O3", "C16H11O3","C15H11O2","C13H11O", "C9H7O3","C10H11[18O]"]
frags=["C19H15O3", "C16H11O3","C15H11O2","C13H11O","C9H7O3","C10H11O"]

In [14]:
from rdkit.Chem.Draw import IPythonConsole

In [15]:
len(filtered)

1058

In [16]:
filtered_no_label=filtered.copy(deep=True)
filtered_O18 = filtered.copy(deep=True)

In [17]:
#replace parallel_apply by progress_apply in windows. But will use 1 core
filtered_O18['mapped_frags'] = filtered_O18.InChI.parallel_apply(calculate_mapped_frags, frags=fragsO18, labeled=True, N_labels=N)

In [18]:
#replace parallel_apply by progress_apply in windows. But wil use 1 core
filtered_no_label['mapped_frags'] = filtered_no_label.InChI.parallel_apply(calculate_mapped_frags, frags=frags, labeled=False, N_labels=N)

In [19]:
a1=filtered_no_label[filtered_no_label.mapped_frags==len(frags)]
a2=filtered_O18[filtered_O18.mapped_frags==len(fragsO18)]

In [21]:
with open ('./results/'+formula+'.txt', 'a') as f:
    f.write('Number of filtered candidaes with matched number of peaks: ' + str(len(a1))+'\n')
    f.write('Number of filtered candidaes with matched number of peaks with O18: '+ str(len(a2))+'\n')

In [22]:
a2.to_csv('./results/final_lists/'+formula+'.csv')